<a href="https://colab.research.google.com/github/FabienMiguel/NLP-Fellowship/blob/main/Fabien_Mbonigaba_2nd_Hackathon_File.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

***Firstly, I do Web Scrapping website to get data I choose(RURA)***

In [11]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

In [ ]:
# This cell is for fetching content from the home page 

url = 'https://rura.rw/index.php'
response = requests.get(url)
html = response.content

soup = BeautifulSoup(html, 'html.parser')
soup


In [ ]:
# This cell is for getting all links in the website

links = soup.find_all('a')
links

In [ ]:
index = -30
link = links[index]['href'] #Get the link
title = links[index].text #Save the description
print('Link:', link, 'Described as:', title)

In [16]:
all_links = [] #list()
list_of_titles = []

for item in links: #loop over the list of links
  if str(item['href'])[0:9] == 'index.php':
    lin = 'https://rura.rw/' + item['href']
  else:
    lin = item['href']

  all_links.append(lin)
  if len(item.text) > 0:
    list_of_titles.append(item.text)
  else:
    list_of_titles.append(None)

In [17]:
df = pd.DataFrame()
df['title'] = list_of_titles
df['link'] = all_links
df['description'] = None

In [ ]:
all_links

In [ ]:
# This cell is for dataframe

df.describe()

In [20]:
# drop all the row with empty title
df = df.dropna(subset=['title']) 
df

,title,link,description
0,business plan to buy an apartment building wri...,#,None
1,business plan to buy an apartment building,http://rura.rw/?business-plan-to-buy-an-apartm...,None
2,writing a good abstract for research paper,http://rura.rw/?writing-a-good-abstract-for-re...,None
3,statement of admission,http://rura.rw/?statement-of-admission,None
10,Home,https://rura.rw/index.php?id=23,None
...,...,...,...
171,Regulatory instruments,https://rura.rw/index.php?id=179,None
172,Sector Reports,https://rura.rw/index.php?id=185,None
173,Statistics,https://rura.rw/index.php?id=86,None
174,FAQs,https://rura.rw/index.php?id=88,None


In [21]:
#Check whehter the link is full formated link
is_full_link = df.link.str.startswith('http')

#Check whether the link is pdf documents
is_pdf = df.link.str.contains('.pdf')

In [ ]:
df['is_full_link'] = is_full_link #Create a new col
df['is_pdf'] = is_pdf #create a new col

df

In [23]:
df = df[df['is_full_link'] == True ] #Drop all the links
df =  df[df['is_pdf'] == False ]

df

,title,link,description,is_full_link,is_pdf
1,business plan to buy an apartment building,http://rura.rw/?business-plan-to-buy-an-apartm...,None,True,False
2,writing a good abstract for research paper,http://rura.rw/?writing-a-good-abstract-for-re...,None,True,False
3,statement of admission,http://rura.rw/?statement-of-admission,None,True,False
10,Home,https://rura.rw/index.php?id=23,None,True,False
11,About RURA,https://rura.rw/index.php?id=25,None,True,False
...,...,...,...,...,...
171,Regulatory instruments,https://rura.rw/index.php?id=179,None,True,False
172,Sector Reports,https://rura.rw/index.php?id=185,None,True,False
173,Statistics,https://rura.rw/index.php?id=86,None,True,False
174,FAQs,https://rura.rw/index.php?id=88,None,True,False


In [24]:
df.iloc[5]

title                                Background
link            https://rura.rw/index.php?id=44
description                                None
is_full_link                               True
is_pdf                                    False
Name: 12, dtype: object

In [25]:
# This cell is for cleanning by using regular expression

import re

def preprocessing(titles : list):
  cleaned_titles = []
  for title in titles:
    title = title.lower()
    url_pattern = re.compile(r'https?://\S+|www\.\S+')
    html_pattern = re.compile('<.*?>')
    title = url_pattern.sub(r'', title)
    title = html_pattern.sub(r'', title)
    title = re.sub(r"[^\w\d'\s]+", ' ', title)
    cleaned_titles.append(title)
  return cleaned_titles


df['title'] = preprocessing(df['title'])


In [ ]:
df

# **Time for Building the Chatbot**


In [8]:
# Asking some questions and Answers!

question = """Guide me to the rura home page?
Guide me to the rura transport page?
where do you get {}?
what is the link to the {}?
how do you get {}?
who is in charge of {}?
etc,... """

Answers = """Here is the link {}.
you can use this link {} for more information."""

In [9]:
question.split('\n')


['Guide me to the rura home page?',
 'Guide me to the rura transport page?',
 'where do you get {}?',
 'what is the link to the {}?',
 'how do you get {}?',
 'who is in charge of {}?',
 'etc,... ']

In [27]:
intents = []

for index in range(len(df)):
  intent = {}
  tag = df.iloc[index]['title']
  intent['tag'] = tag
  intent['patterns'] = list()
  intent['responses'] = [f'You can use this link {df.iloc[index].link} for more information']
  for i in range(len(question.split('\n'))):
    intent['patterns'].append(question.split('\n')[i].format(intent['tag']))

  intents.append(intent)
  
file = {'intents': intents}

In [ ]:
file

In [3]:
#!git clone https://github.com/FabienMiguel/chattbot.git

Cloning into 'chattbot'...
remote: Enumerating objects: 30, done.
remote: Counting objects: 100% (30/30), done.
remote: Compressing objects: 100% (21/21), done.
remote: Total 30 (delta 6), reused 30 (delta 6), pack-reused 0
Unpacking objects: 100% (30/30), done.


In [29]:
import json

In [31]:
with open('/content/chattbot/intents.json', 'w+') as f:
  f.seek(0)
  f.truncate()
  json.dump(file, f)

In [33]:
%cd /content/chattbot

/content/chattbot


In [34]:
!ls

app.py	  intents.json	 __pycache__	      static	 train.py
chat.py   model.py	 README.md	      st.py
data.pth  nltk_utils.py  standalone-frontend  templates


In [35]:
#This cell is for tokenizetion of data that why i've to import nltk library

import nltk 
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

# **Time to Train the Chatbot to connect the Patterns and Specific questions** 


In [36]:
#This cell is for installing file to train

!python train.py

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
1155 patterns
89 tags: ['\r\n          an ai ethics primer for the rwandan ai ecosystem and consultation on ethical ai guidelines \r\n        ', '\r\n          open call for the rwandan artificial intelligence ai  ecosystem  ai ethics applied programme and consultation\r\n        ', '\r\n          rura and zambian energy regulator exchange experience on lpg regulation\r\n        ', '\r\n          rura hosts government institutions to discuss nuclear safeguards implementation\r\n        ', '\r\n          rura in partnership with iaea hosts a regional workshop on radiation protection officers and qualified experts\r\n        ', 'about rura', 'all online services', 'announcements', 'annual reports', 'background', 'board decisions', 'business plan to buy an apartment building', 'complaints handling procedures', 'consumer affairs', 'consumer awareness and educational tools', 'consu

In [37]:
#This cell is for chat

!python chat.py

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
Let's chat! (type 'quit' to exit)
You: guide me about rura?
You can use this link https://rura.rw/index.php?id=25 for more information
You: guide me to the rura transport page?
I do not understand...
You: quit


In [38]:
import random
import json

import torch

from model import NeuralNet
from nltk_utils import bag_of_words, tokenize

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

with open('intents.json', 'r') as json_data:
    intents = json.load(json_data)

FILE = "data.pth"
data = torch.load(FILE)

input_size = data["input_size"]
hidden_size = data["hidden_size"]
output_size = data["output_size"]
all_words = data['all_words']
tags = data['tags']
model_state = data["model_state"]

model = NeuralNet(input_size, hidden_size, output_size).to(device)
model.load_state_dict(model_state)
model.eval()

bot_name = "Jery"
print("Let's have a fun!! (type 'quit' to exit)")

def query(question : str):
  # sentence = "do you use credit cards?"
  sentence = question

  sentence = tokenize(sentence)
  X = bag_of_words(sentence, all_words)
  X = X.reshape(1, X.shape[0])
  X = torch.from_numpy(X).to(device)

  output = model(X)
  _, predicted = torch.max(output, dim=1)

  tag = tags[predicted.item()]

  probs = torch.softmax(output, dim=1)
  prob = probs[0][predicted.item()]
  if prob.item() > 0.75:
      for intent in intents['intents']:
          if tag == intent["tag"]:
              return f"{bot_name}: {random.choice(intent['responses'])}"
  else:
      return f"{bot_name}: I do not understand..."

Let's have a fun!! (type 'quit' to exit)


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [39]:
response = query('tell me about rura')
response

'Jery: You can use this link https://rura.rw/index.php?id=25 for more information'

In [40]:
!pip install langdetect

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 981 kB 5.0 MB/s 
  Created wheel for langdetect: filename=langdetect-1.0.9-py3-none-any.whl size=993242 sha256=2b01aa5f434f01e0727c123faa7c2e1d46b03b90bdf6684bf39c68ab4028b02b
  Stored in directory: /root/.cache/pip/wheels/13/c7/b0/79f66658626032e78fc1a83103690ef6797d551cb22e56e734
Successfully built langdetect


In [41]:

import requests
import json


class translator:
    api_url = "https://translate.googleapis.com/translate_a/single"
    client = "?client=gtx&dt=t"
    dt = "&dt=t"

    #fROM English to Kinyarwanda
    def translate(text : str , target_lang : str, source_lang : str):
        sl = f"&sl={source_lang}"
        tl = f"&tl={target_lang}"
        r = requests.get(translator.api_url+ translator.client + translator.dt + sl + tl + "&q=" + text)
        return json.loads(r.text)[0][0][0]

In [42]:
resp = translator.translate(text='Mpereza amakuru yerekeye LeoMessi', target_lang='fr', source_lang='rw')
resp

"Je termine l'actualité de LeoMessi"

In [43]:
from langdetect import detect

def process_question(text : str):

  source_lang = detect(text)
  resp = translator.translate(text=text, target_lang='en', source_lang=source_lang)
  return resp, source_lang

def process_answer(text : str, source_lang):
  resp = translator.translate(text=text, target_lang=source_lang, source_lang='en')
  return resp

In [44]:
def process(QUESTION: str):
  USER_QUERY, SL = process_question(QUESTION) #Translate the original question into english and store the source lang
  RESPONSE = query(USER_QUERY) #Asking th chatbot question
  ORIGINAL_RESPONSE = process_answer(RESPONSE, SL)
  return ORIGINAL_RESPONSE

process("Donne mois de l'information sur RURA?")

"Jery \u200b\u200b: Vous pouvez utiliser ce lien https://rura.rw/index.php?id=25 pour plus d'informations"

 **Time for FastAPI**

In [45]:
!pip install fastapi pyngrok uvicorn nest-asyncio

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 55 kB 2.3 MB/s 
     |████████████████████████████████| 761 kB 10.4 MB/s 
     |████████████████████████████████| 56 kB 5.6 MB/s 
     |████████████████████████████████| 64 kB 3.0 MB/s 
     |████████████████████████████████| 80 kB 9.6 MB/s 
     |████████████████████████████████| 58 kB 6.2 MB/s 
  Created wheel for pyngrok: filename=pyngrok-5.2.1-py3-none-any.whl size=19792 sha256=9ed21fa8fe28cea70041a12a2aebacc33f14777c45adb45ddbe2a96f47224fc2
  Stored in directory: /root/.cache/pip/wheels/5d/f2/70/526da675d32f17577ec47ac4c663084efe39d47c826b6c3bb1
Successfully built pyngrok


In [46]:
!ngrok authtoken 2HJQ5KFUMPejTBPPFOQGP2WRU74_o2vYuu2ZQcGb3zYD6ziq

Authtoken saved to configuration file: /root/.ngrok2/ngrok.yml


In [47]:
from fastapi import FastAPI, Request
from fastapi.responses import HTMLResponse

In [48]:
#Cell for chatbot title
app = FastAPI(title='Hold_Up Chatbot')

In [49]:
@app.get('/')
def index():
  return 'Welcome to my Hold_Up Web app'


@app.get('/chatbot')
def chatbot(query : str = None):
  return process(query)

In [ ]:
import nest_asyncio
from pyngrok import ngrok
import uvicorn

ngrok_tunnel = ngrok.connect(8000)
print('Please use the following link to access you app', ngrok_tunnel.public_url)

nest_asyncio.apply()
uvicorn.run(app, port=8000)

INFO:     Started server process [74]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://127.0.0.1:8000 (Press CTRL+C to quit)


Please use the following link to access you app http://eade-34-75-124-181.ngrok.io
INFO:     2c0f:eb68:203:3c00:802f:c800:be67:7648:0 - "GET / HTTP/1.1" 200 OK
INFO:     2c0f:eb68:203:3c00:802f:c800:be67:7648:0 - "GET /favicon.ico HTTP/1.1" 404 Not Found


Cloning into 'chatbot-deployment-patrickloeber-'...
remote: Enumerating objects: 37, done.
remote: Counting objects: 100% (35/35), done.
remote: Compressing objects: 100% (23/23), done.
remote: Total 37 (delta 9), reused 32 (delta 7), pack-reused 2
Unpacking objects: 100% (37/37), done.
